# Frequency Modulation

In [ ]:
from disiple.signals import AudioSignal, TimeSignal, Spectrum
from disiple.util import nextpow2
import numpy as np
from scipy.signal import hilbert, sawtooth
from bokeh.plotting import show
from bokeh.layouts import row, column, gridplot
from bokeh.models import CustomJS, Slider

In [ ]:
samplerate = 22050
duration = nextpow2(samplerate) / samplerate # at least 1 second
times = np.arange(0, duration, 1/samplerate)

In [ ]:
mod_idx = 3

### Modulating Signal

In [ ]:
amp_mod = 1
freq_mod = 220/duration
# samples_mod = amp_mod * np.sin(2*np.pi*freq_mod*times) # FM & PM only differ in phase for sinusoidal messages
samples_mod = np.clip(3*amp_mod*sawtooth(2*np.pi*freq_mod*times, width=0.5), -amp_mod, amp_mod)
mod_signal = AudioSignal(samples_mod, samplerate)
mod_spec = Spectrum.from_timesignal(mod_signal, dB=False)

### Carrier Signal

In [ ]:
amp_carr = 1
freq_carr = 2600
samples_carr = amp_carr * np.sin(2*np.pi*freq_carr*times)
carr_signal = AudioSignal(samples_carr, samplerate)
carr_spec = Spectrum.from_timesignal(carr_signal, dB=False)

### Frequency Modulation

In [ ]:
samples_integral_mod = 2*np.pi*freq_mod * np.cumsum(samples_mod) / samplerate
integral_mod_signal = TimeSignal(samples_integral_mod, samplerate)

In [ ]:
samples_fm = amp_carr * np.sin(2*np.pi*freq_carr*times + mod_idx * samples_integral_mod)
fm_signal = AudioSignal(samples_fm, samplerate)
fm_spec = Spectrum.from_timesignal(fm_signal, dB=False)
# sensitivity_fm = mod_idx * freq_mod / amp_mod

### Phase Modulation

In [ ]:
samples_pm = amp_carr * np.sin(2*np.pi*freq_carr*times + mod_idx * samples_mod)
pm_signal = AudioSignal(samples_pm, samplerate)
pm_spec = Spectrum.from_timesignal(pm_signal, dB=False)
# sensitivity_pm = mod_idx / amp_mod

### Frequency Demodulation

In [ ]:
inst_angle = hilbert(samples_fm) * np.exp(2*np.pi*freq_carr*times*-1j)
samples_fm_demod = np.diff(np.unwrap(np.angle(inst_angle))) * samplerate / (2*np.pi*mod_idx*freq_mod)
samples_fm_demod = np.concatenate((samples_fm_demod[:1], samples_fm_demod))
fm_demod_signal = AudioSignal(samples_fm_demod, samplerate)
defm_spec = Spectrum.from_timesignal(fm_demod_signal, dB=False)

### Phase Demodulation

In [ ]:
init_phase = np.arccos(samples_carr[0]/amp_carr)
inst_angle = hilbert(samples_pm) * np.exp((2*np.pi*freq_carr*times - init_phase)*-1j)
samples_pm_demod = np.angle(inst_angle) / mod_idx
pm_demod_signal = AudioSignal(samples_pm_demod, samplerate)
depm_spec = Spectrum.from_timesignal(pm_demod_signal, dB=False)

### Create Figures

In [ ]:
time_range = (0, 6/freq_mod) # show first 6 periods of signal = 6 * 1/freq_mod
mag_range = (0, 0.65)

mod_fig = mod_signal.plot(title='Message Signal', x_range=time_range, active_inspect=None)
mod_spec_fig = mod_spec.plot(title='Magnitude Spectrum of Message Signal', y_range=mag_range, active_inspect=None)
carr_fig = carr_signal.plot(title='Carrier Signal', x_range=time_range, active_inspect=None, line_color='olive')
carr_spec_fig = carr_spec.plot(title='Magnitude Spectrum of Carrier Signal', y_range=mag_range, active_inspect=None, line_color='olive')

fm_fig = fm_signal.plot(title='Frequency Modulated Signal', legend_label='FM Signal', x_range=time_range, active_inspect=None, line_color='crimson')
(amp_carr*mod_signal).plot(fig=fm_fig, legend_label='Message Signal')
fm_fig.legend.location = 'bottom_right'
fm_fig.legend.click_policy = 'hide'
fm_spec_fig = fm_spec.plot(title='Magnitude Spectrum of FM Signal', y_range=mag_range, active_inspect=None, line_color='crimson')

pm_fig = pm_signal.plot(title='Phase Modulated Signal', legend_label='PM Signal', x_range=time_range, active_inspect=None, line_color='purple')
(amp_carr*mod_signal).plot(fig=pm_fig, legend_label='Message Signal')
pm_fig.legend.location = 'bottom_right'
pm_fig.legend.click_policy = 'hide'
pm_spec_fig = pm_spec.plot(title='Magnitude Spectrum of PM Signal', y_range=mag_range, active_inspect=None, line_color='purple')

fm_demod_fig = fm_demod_signal.plot(x_range=time_range, line_color='crimson', legend_label='Reconstructed Message of FM')
fm_demod_fig.legend.location = 'bottom_right'
fm_demod_fig.legend.click_policy = 'hide'
mod_signal.plot(fig=fm_demod_fig, legend_label='Original Message', visible=False)
defm_spec_fig = defm_spec.plot(title='Magnitude Spectrum of Reconstructed Message Signal of FM', y_range=mag_range, active_inspect=None, line_color='crimson')

pm_demod_fig = pm_demod_signal.plot(x_range=time_range, line_color='purple', legend_label='Reconstructed Message of PM')
pm_demod_fig.legend.location = 'bottom_right'
pm_demod_fig.legend.click_policy = 'hide'
mod_signal.plot(fig=pm_demod_fig, legend_label='Original Message', visible=False)
depm_spec_fig = depm_spec.plot(title='Magnitude Spectrum of Reconstructed Message Signal of PM', y_range=mag_range, active_inspect=None, line_color='purple')

### Link Time and Frequency Axes

In [ ]:
from itertools import product
def link_x_axes(figs):
    for fig1, fig2 in product(figs, figs):
        fig1.x_range.js_link('start', fig2.x_range, 'start')
        fig1.x_range.js_link('end', fig2.x_range, 'end')
link_x_axes({mod_fig, carr_fig, fm_fig, pm_fig, fm_demod_fig, pm_demod_fig})
link_x_axes({mod_spec_fig, carr_spec_fig, fm_spec_fig, pm_spec_fig, defm_spec_fig, depm_spec_fig})

### Display Figures

In [ ]:
show(row(gridplot([mod_fig, carr_fig, fm_fig, pm_fig, fm_demod_fig, pm_demod_fig], ncols=1, width=700), gridplot([mod_spec_fig, carr_spec_fig, fm_spec_fig, pm_spec_fig, defm_spec_fig, depm_spec_fig], ncols=1, width=700)))

### Add Interaction

In [ ]:
min_freq_mod, max_freq_mod = freq_mod / 2, 2 * freq_mod
mod_idx_slider = Slider(start=0, end=10, value=mod_idx, step=.01, title='Modulation Index')
amp_mod_slider = Slider(start=0, end=1, value=amp_mod, step=.01, title='Message Amplitude')
freq_mod_slider = Slider(start=min_freq_mod, end=max_freq_mod, value=freq_mod, step=10, title='Message Frequency')
amp_carr_slider = Slider(start=0, end=1, value=amp_carr, step=.01, title='Carrier Amplitude')
freq_carr_slider = Slider(start=1000, end=min(4000, samplerate/2-max_freq_mod), value=freq_carr, step=100, title='Carrier Frequency')

callback = CustomJS(args=dict(modSource=mod_fig.renderers[0].data_source,
                              carrSource=carr_fig.renderers[0].data_source,
                              fmSource=fm_fig.renderers[0].data_source,
                            #   fmModSource=fm_fig.renderers[1].data_source,
                              pmSource=pm_fig.renderers[0].data_source,
                            #   pmModSource=pm_fig.renderers[1].data_source,
                              modSpecSource=mod_spec_fig.renderers[0].data_source,
                              carrSpecSource=carr_spec_fig.renderers[0].data_source,
                              fmSpecSource=fm_spec_fig.renderers[0].data_source,
                              pmSpecSource=pm_spec_fig.renderers[0].data_source,
                              modSamples=samples_mod,
                              samplerate=samplerate,
                              modIdxSlider = mod_idx_slider,
                              # ampModSlider=amp_mod_slider,
                              ampMod = amp_mod,
                              # freqModSlider=freq_mod_slider,
                              freqMod = freq_mod,
                              ampCarrSlider=amp_carr_slider,
                              freqCarrSlider=freq_carr_slider,
                             ), code="""
    const modIdx = modIdxSlider.value;
    //const ampMod = ampModSlider.value;
    const ampCarr = ampCarrSlider.value;
    //const freqMod = freqModSlider.value;
    const freqCarr = freqCarrSlider.value;

    //const modSamples = modSource.data.x.map((t) => ampMod * Math.sin(2*Math.PI*freqMod*t));
    //modSource.data = {'x': modSource.data.x, 'y': modSamples};
    //const scaledModSamples = modSamples.map((t) => t * ampCarr);
    //fmModSource.data = {'x': fmModSource.data.x, 'y': scaledModSamples};
    //pmModSource.data = {'x': pmModSource.data.x, 'y': scaledModSamples};
    const carrSamples = carrSource.data.x.map((t) => ampCarr * Math.sin(2*Math.PI*freqCarr*t));
    carrSource.data = {'x': carrSource.data.x, 'y': carrSamples};

    const cumSum = modSamples.map((sum => value => sum += value)(0));
    const fmSamples = fmSource.data.x.map((t, idx) => ampCarr * Math.sin(2*Math.PI*freqCarr*t + modIdx * 2*Math.PI*freqMod*cumSum[idx]/samplerate));
    fmSource.data = {'x': fmSource.data.x, 'y': fmSamples};
    const pmSamples = pmSource.data.x.map((t, idx) => ampCarr * Math.sin(2*Math.PI*freqCarr*t + modIdx * modSamples[idx]));
    pmSource.data = {'x': pmSource.data.x, 'y': pmSamples};

    const freqs = fourier.rfftFreqs(samplerate);
    modSpecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.magnitudeSpectrum(modSamples)};
    carrSpecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.magnitudeSpectrum(carrSamples)};
    fmSpecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.magnitudeSpectrum(fmSamples)};
    pmSpecSource.data = {'bin_unit': freqs, 'frequency': freqs, 'magnitude': fourier.magnitudeSpectrum(pmSamples)};
""")
mod_idx_slider.js_on_change('value', callback)
amp_mod_slider.js_on_change('value', callback)
freq_mod_slider.js_on_change('value', callback)
amp_carr_slider.js_on_change('value', callback)
freq_carr_slider.js_on_change('value', callback)

interative_plot = column(
    row(gridplot([mod_fig, carr_fig], ncols=1, width=700), gridplot([mod_spec_fig, carr_spec_fig], ncols=1, width=700)),
    row(mod_idx_slider, amp_carr_slider, freq_carr_slider),
    row(gridplot([fm_fig, pm_fig], ncols=1, width=700), gridplot([fm_spec_fig, pm_spec_fig], ncols=1, width=700))
)
show(interative_plot)

In [ ]:
from bokeh.plotting import save
from bokeh.resources import INLINE
template = f"""
{{% block postamble %}}
    <script src="fourier.js"></script>
    <script>const fourier = new Fourier({len(times)});</script>
{{% endblock %}} """
save(interative_plot, filename='fm-pm.html', title='Frequency & Phase Modulation', resources=INLINE, template=template)